In [1]:
from IPython.display import display, Image

# CNTK 103: Part D - Сверточная нейронная сеть с MNIST

Мы предполагаем, что вы изучили CNTK 103 Part A CNTK 103 Part A (MNIST Data Loader).

В этом уроке мы будем тренировать сверточную нейронную сеть (CNN) по данным MNIST. Используется API Python. Если вы ищете этот пример в BrainScript, посмотрите(https://github.com/Microsoft/CNTK/tree/release/2.2/Examples/Image/GettingStarted)

## Введение

[Сверточная нейронная сеть](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN или ConvNet) является типом [feed-forward](https://en.wikipedia.org/wiki/Feedforward_neural_network) искусственная нейронная сеть, состоящая из нейронов, обладающих узнаваемыми весами и смещениями, очень похожей на обычные многослойные сети персептрона (MLP), введенные в 103C. CNN используют пространственный характер данных. В природе мы воспринимаем разные объекты по их форме, размеру и цвету. Например, объекты  обычно представляют собой ребра, углы / вершины (определенные двумя и более краями), цветовые пятна и т. Д. Эти примитивы часто идентифицируются с использованием разных детекторов (например, детектирование края, детектор цвета) или комбинация взаимодействующих детекторов для облегчения интерпретации изображений (классификация объектов, определение области, описание и т. д.) в задачах, связанных с реальным миром. Эти детекторы также известны как фильтры. Convolution - это математический оператор, который принимает изображение и фильтр в качестве входных данных и производит отфильтрованный вывод  (представляющий собой  углы, цвета и т. д. во входном изображении).  Исторически эти фильтры представляют собой набор весов, которые часто обрабатывались вручную или моделировались математическими функциями (например [Gaussian](https://en.wikipedia.org/wiki/Gaussian_filter) / [Laplacian](http://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm) / [Canny](https://en.wikipedia.org/wiki/Canny_edge_detector) фильтрами).  Выходы фильтра отображаются через функции нелинейной активации, имитирующие человеческие клетки мозга, называемые [нейронами](https://en.wikipedia.org/wiki/Neuron).

Сверточные сети предоставляют механизм для изучения этих фильтров непосредственно из данных вместо явных математических моделей и, как было установлено, превосходят (в реальных задачах) по сравнению с исторически создаваемыми фильтрами. В сверточных сетях основное внимание уделяется изучению весов фильтров, а не обучению индивидуально полностью подключенных пар (между входами и выходами) весов. Таким образом, количество весов для обучения уменьшается по сравнению с традиционными сетями MLP из предыдущих уроков. В сверточной сети обочается несколько фильтров, от нескольких отдельных цифр до нескольких тысяч в зависимости от сложности сети.

Было показано, что многие из примитивов CNN имеют концептуально параллельные компоненты в зрительной коре головного мозга  [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex). Группа нейронов в зрительной коре выделяет ответы при стимуляции. Эта область известна как восприимчивое поле  (RF).  Эквивалентно, в сверточной сети область ввода, соответствующая размерам фильтра, может рассматриваться как восприимчивое поле. Глубокие  CNNs или ConvNets сети (такие как [AlexNet](https://en.wikipedia.org/wiki/AlexNet), [VGG](https://arxiv.org/abs/1409.1556), [Inception](http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf), [ResNet](https://arxiv.org/pdf/1512.03385v1.pdf)) используются для различных [computer vision](https://en.wikipedia.org/wiki/Computer_vision) задач.  

В этом уроке мы используем операцию свертки и познакомимся с различными параметрами в CNN.

**Проблема**:
Как и в CNTK 103C, мы продолжим работу над той же проблемой распознавания цифр в данных MNIST. Данные MNIST содержат рукописные цифры с небольшим фоновым шумом.

In [1]:
# Рисунок 1
Image(url= "http://3.bp.blogspot.com/_UpN7DfJA0j4/TJtUBWPk0SI/AAAAAAAAABY/oWPMtmqJn3k/s1600/mnist_originals.png", width=200, height=200)

NameError: name 'Image' is not defined

**Цель**:
Наша цель - подготовить классификатор, который будет идентифицировать цифры в наборе данных MNIST. 

**Подход**:

Используются те же 5 этапов, которые мы использовали в предыдущем учебнике: чтение данных, предварительная обработка данных, создание модели, изучение параметров модели и оценка  (a.k.a. testing/prediction) модели. 
- Чтение данных: мы будем использовать CNTK Text reader 
- Предварительная обработка данных: Поясняется в части A.

В этом уроке мы будем экспериментировать с двумя моделями с различными архитектурными компонентами.

In [3]:
from __future__ import print_function # Используйте определение функции  версии (скажем 3.x from 2.7 interpreter)
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time

import cntk as C
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (требуется только для нашей системы сборки)
C.cntk_py.set_fixed_random_seed(1) # перемешаем, чтобы примеры не повторялись

%matplotlib inline

ModuleNotFoundError: No module named 'cntk'

## Чтение данных

В этом разделе мы используем данные, сгенерированные в CNTK 103 Part A (MNIST Data Loader).

Мы используем данные MNIST, загруженные с помощью ноутбука  CNTK_103A_MNIST_DataLoader. . Набор данных содержит 60 000 обучающих изображений и 10 000 тестовых изображений, каждое изображение которых составляет 28 х 28 пикселей. Таким образом, количество функций равно 784 (= 28 x 28 пикселей), 1 на пиксель. Переменная  `num_output_classes` установлена в 10, соответствующая количеству цифр (0-9) в наборе данных.

В предыдущих уроках, как показано ниже, мы всегда сглаживали входное изображение в вектор. С сверточными сетями мы не сглаживаем изображение таким образом.

![](https://www.cntk.ai/jup/cntk103a_MNIST_input.png)

**Размеры входного сигнала**:  

В сверточных сетях для изображений входные данные часто формируются как трехмерная матрица (количество каналов, ширина изображения, высота), что сохраняет пространственную взаимосвязь между пикселями. На рисунке выше изображение MNIST представляет собой данные с одним каналом (в оттенках серого), поэтому размер ввода задается как (1, ширина изображения, высота изображения).

![](https://www.cntk.ai/jup/cntk103d_rgb.png)

Естественные цветные изображения часто представлены в цветовых каналах Red-Green-Blue (RGB). Входной размер таких изображений задается как  (3, ширина изображения, высота изображения). Если входные данные RGB являются объемным сканированием с шириной объема, объемом и глубиной объема, представляющей 3 оси, формат входных данных будет определяться  из 4 значений (3, объемная ширина, высота объема, глубина объема). Таким образом, CNTK позволяет специфицировать входные изображения в произвольном многомерном пространстве.

In [4]:
# Определение размеров данных
input_dim_model = (1, 28, 28)    # изображения 28 x 28 с 1 каналом цвета (серый)
input_dim = 28*28                # используемые читателями для обработки входных данных в виде вектора
num_output_classes = 10

**Формат данных** Данные хранятся на нашей локальной машине в формате CNTK CTF. Формат CTF представляет собой простой текстовый формат, который содержит набор выборок с каждым образцом, содержащим набор именованных полей и их данных. Для наших данных MNIST каждый образец содержит 2 поля: метки и функции, отформатированные как:

    |labels 0 0 0 1 0 0 0 0 0 0 |features 0 255 0 123 ... 
                                                  (784 integers each representing a pixel gray level)
    
В этом уроке мы будем использовать пиксели изображения, соответствующие целочисленному потоку с именем "features".  Мы определяем функцию  `create_reader` для чтения данных обучения и тестирования с помощью [CTF deserializer](https://cntk.ai/pythondocs/cntk.io.html#cntk.io.CTFDeserializer).

Метки [1-hot](https://en.wikipedia.org/wiki/One-hot) зокодированы (метка, представляющая класс вывода 3, становится `0001000000` , так как у нас есть 10 классов для 10 возможных цифр) , где первый индекс соответствует цифре `0` , а последний соответствует цифре `9`.

![](https://www.cntk.ai/jup/cntk103a_onehot.png)

In [5]:
# Прочтите текст в формате CTF (как упоминалось выше), используя десериализатор CTF из файла
def create_reader(path, is_training, input_dim, num_label_classes):
    
    ctf = C.io.CTFDeserializer(path, C.io.StreamDefs(
          labels=C.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False),
          features=C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)))
                          
    return C.io.MinibatchSource(ctf,
        randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [6]:
# Убедитесь, что доступны учебные и тестовые данные.
# Мы выполняем поиск в двух местах в наборе инструментов для кэшированного набора данных MNIST.

data_found=False # Флаг, указывающий, обнаружены ли данные train/test data found  в локальном кеше
for data_dir in [os.path.join("..", "Examples", "Image", "DataSets", "MNIST"),
                 os.path.join("data", "MNIST")]:
    
    train_file=os.path.join(data_dir, "Train-28x28_cntk_text.txt")
    test_file=os.path.join(data_dir, "Test-28x28_cntk_text.txt")
    
    if os.path.isfile(train_file) and os.path.isfile(test_file):
        data_found=True
        break
        
if not data_found:
    raise ValueError("Please generate the data by completing CNTK 103 Part A")
    
print("Data directory is {0}".format(data_dir))

Data directory is data\MNIST


## Создание модели CNN

CNN - это прямая сеть, состоящая из множества слоев таким образом, что выход одного слоя становится входным для следующего слоя (аналогично MLP). В MLP все возможные пары входных пикселей соединены с выходными узлами и с каждой парой, имеющей вес, что приводит к комбинаторному изменению параметров, подлежащих изучению, а также к увеличению возможности переобучения ([детали](http://cs231n.github.io/neural-networks-1/)).  Уровни свертки используют пространственное расположение пикселей и изучают несколько фильтров, которые значительно уменьшают количество параметров в сети ([детали](http://cs231n.github.io/convolutional-networks/)). Размер фильтра является параметром слоя свертки. 

В этом разделе мы введем основы операций свертки. Мы показываем иллюстрации в контексте RGB-изображений (3 канала), хотя данные MNIST, которые мы используем в этом учебнике, представляют собой изображение в оттенках серого (один канал).

![](https://www.cntk.ai/jup/cntk103d_rgb.png)

### Сверточный слой

Слой свертки представляет собой набор фильтров. Каждый фильтр определяется весовой (**W**) матрицей и смещением ($b$).

![](https://www.cntk.ai/jup/cntk103d_filterset_v2.png)

Эти фильтры сканируются через изображение, выполняющее точечный проход между весами и соответствующим входным значением ($x$). Значение смещения добавляется к выходу точечного произведения, и полученная сумма необязательно отображается через функцию активации. Этот процесс проиллюстрирован в следующей анимации.

In [7]:
Image(url="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif", width= 300)

NameError: name 'Image' is not defined

Уровни свертки включают следующие основные функции:

   - Вместо того, чтобы полностью подключаться ко всем парам входных и выходных узлов, каждый узел свертки **локально подключенного - locally-connected** к подмножеству входных узлов, локализованных в меньшей области ввода, также называемой восприимчивым полем (RF). На рисунке выше показаны небольшие 3 x 3 области изображения в качестве области сканирования. В случае с RGB изображением будет иметь три таких 3 x 3 области, по одному из трех цветовых каналов. 
   
   
   - Вместо того, чтобы иметь один набор весов (как в плотном слое), сверточные слои имеют несколько наборов (показаны на рисунке с несколькими цветами), называемые **фильтрами - filters**. Каждый фильтр обнаруживает функции в каждом возможном RF во входном изображении. Вывод свертки представляет собой набор подслоев `n`(показано в анимации ниже), где `n` количество фильтров (см. Приведенный выше рисунок).  
   
     
   - Внутри подуровня вместо каждого узла, имеющего собственный набор весов, один набор **общих весов - shared weights**  используется всеми узлами в этом подуровне. Это уменьшает количество параметров, подлежащих изучению, и, таким образом, перерабатывает. Это также открывает двери для нескольких аспектов глубокого обучения, которые позволили построить очень практичные решения:
    - Обработка больших изображений (например, 512 x 512)
    - Попытка увеличить размер фильтра (соответствует более крупному RF), например, 11 x 11
    - Изучение большего количества фильтров (скажем 128)
    - Исследование более глубоких архитектур (более 100 слоев)
    - Обеспечить переводную инвариантность (возможность распознавания функции независимо от того, где они появляются на изображении). 

### Параметры Strides и Pad

**Как размещаются фильтры?** В общем, фильтры расположены в перекрывающихся областях слева направо и сверху вниз. Каждый уровень свертки имеет параметр для указания `filter_shape`, определяющий ширину и высоту фильтра в случае большинства естественных изображений. Существует параметр (`strides`) , который управляет тем, как далеко перейти вправо при перемещении фильтров через несколько RF-строк в ряд, и как далеко отступить при переходе к следующей строке. Логический параметр `pad` управляет, если вход должен быть дополнен по краям, чтобы обеспечить полную разбивку RF на границах. 

Анимация выше показывает результаты с помощью `filter_shape` = (3, 3), `strides` = (2, 2) и `pad` = False. Две приведенные ниже анимации показывают результаты, когда для параметра `pad` становлено значение True. Во-первых, с шагом 2 и вторым с шагом 1.

Примечание: форма вывода (the teal layer)  отличается между двумя настройками шага. Множество ваших решений о выборе площадки и значения шага  основаны на форме требуемого уровня вывода.

In [8]:
# Графические изображения с шагом 2 и 1 с включенным дополнением
images = [("https://www.cntk.ai/jup/cntk103d_padding_strides.gif" , 'With stride = 2'),
          ("https://www.cntk.ai/jup/cntk103d_same_padding_no_strides.gif", 'With stride = 1')]

for im in images:
    print(im[1])
    display(Image(url=im[0], width=200, height=200))

With stride = 2


NameError: name 'display' is not defined

## Построение наших моделей CNN

В этом учебнике CNN мы сначала определяем два контейнера. Один для входного изображения MNIST, а второй - метки, соответствующие 10 цифрам. При чтении данных считыватель автоматически отображает 784 пикселя на изображение в форму, определенную  `input_dim_model` (в этом примере она установлена (1, 28, 28)).

In [9]:
x = C.input_variable(input_dim_model)
y = C.input_variable(num_output_classes)

NameError: name 'C' is not defined

Первая построенная нами модель представляет собой простую сеть сверток. Здесь мы имеем два сверточных слоя. Поскольку наша задача - обнаружить 10 цифр в базе данных MNIST, выход сети должен быть вектором длиной 10, 1, соответствующим каждой цифре. Это достигается путем проецирования вывода последнего сверточного слоя с использованием плотного слоя с выходом  `num_output_classes`. Мы видели это раньше с логистической регрессией и MLP, где функции были сопоставлены с количеством классов в конечном слое. Также обратите внимание, что поскольку мы будем использовать операцию `softmax` которая комбинируется с функцией потери `cross entropy` во время тренировки (см. Несколько ячеек ниже), последний плотный слой не имеет связанной с ним функции активации.

На следующем рисунке показана модель, которую мы собираемся построить. Обратите внимание, что параметры в приведенной ниже модели должны быть экспериментированы. Они часто называют сетевыми гиперпараметрами. Увеличение формы фильтра приводит к увеличению количества параметров модели, увеличивает время вычисления и помогает модели лучше соответствовать данным. Тем не менее, существует риск [overfitting - переобучения](https://en.wikipedia.org/wiki/Overfitting).  Как правило, количество фильтров в более глубоких слоях больше, чем количество фильтров в слоях перед ними. Мы выбрали 8, 16 для первого и второго слоев соответственно. С этими гиперпараметрами следует экспериментировать во время построения модели.

![](https://www.cntk.ai/jup/cntk103d_convonly2.png)

In [10]:
# функция построения модели

def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
            h = features
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(2,2), 
                                       pad=True, name='first_conv')(h)
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=16, 
                                       strides=(2,2), 
                                       pad=True, name='second_conv')(h)
            r = C.layers.Dense(num_output_classes, activation=None, name='classify')(h)
            return r

Создадим экземпляр модели и проверим различные компоненты модели. `z` будет использоваться для представления вывода сети. В этой модели мы используем функцию активации `relu` . 
Примечание: использование `C.layers.default_options`  - это элегантный способ создания сжатых моделей. Это ключ к минимизации ошибок моделирования, что экономит драгоценное время отладки.

In [11]:
# Создать модель
z = create_model(x)

# Печать выходных shapes / parameters различных компонентов
print("Output Shape of the first convolution layer:", z.first_conv.shape)
print("Bias value of the last dense layer:", z.classify.b.value)

NameError: name 'x' is not defined

Понимание числа параметров модели, которые должны быть оценены, является ключом к глубокому обучению, поскольку существует прямая зависимость от объема данных, который требуется иметь. Вам нужно больше данных для модели, которая имеет большее количество параметров для предотвращения переобучения. Другими словами, при фиксированном объеме данных необходимо ограничить количество параметров. Между объемом данных, требуемым для модели, нет золотого правила. Тем не менее, есть способы повысить производительность обучения модели с помощью [data augmentation](https://deeplearningmania.quora.com/The-Power-of-Data-Augmentation-2). 

In [12]:
# Количество параметров в сети
C.logging.log_number_of_parameters(z)

NameError: name 'C' is not defined

**Понимание параметров**:


Наша модель имеет 2 слоя свертки, каждый из которых имеет вес и смещение. Это добавляет до 4 тензоров параметров. Кроме того, плотный слой имеет тензоры веса и смещения. Таким образом, 6 параметрических тензоров.

Давайте теперь подсчитаем количество параметров:
- *Первый слой свертки*: Tимеется 8 фильтров каждого размера (1 x 5 x 5), где 1 - количество каналов на входном изображении. Это добавляет до 200 значений в весовой матрице и 8 значений смещения.


- *Второй уровень свертки*: имеется 16 фильтров каждого размера (8 x 5 x 5), где 8 - количество каналов на входе ко второму слою (= выход первого слоя). Это добавляет до 3200 значений в весовой матрице и 16 значений смещения.


- *Последний плотный слой*: есть 16 x 7 x 7 входных значений, и он производит 10 выходных значений, соответствующих 10 цифрам в наборе данных MNIST. Это соответствует (16 x 7 x 7) x 10 весовых значений и 10 значений смещения.

Что дает модели 11274 параметров.

**Проверка знаний**: Соответствует ли плотная форма слоя задаче (классификация цифр MNIST)?

** Самостоятельная работа **
- Попробуйте изменить формы и параметры различных сетевых слоев,
- Запишите ошибку обучения, которую вы получите с помощью функции `relu` в качестве функции активации,
- Теперь измените на `sigmoid` ак функцию активации и посмотрите, можете ли вы улучшить свою ошибку обучения.

*Дополнительно*: Различные поддерживаемые функции активации [взять здесь](https://cntk.ai/pythondocs/cntk.layers.layers.html#cntk.layers.layers.Activation). Какая функция активации дает наименьшую ошибку обучения?

### Параметры модели обучения

Как и в предыдущем учебнике, мы используем функциюe `softmax` для сопоставления накопленных доказательств или активации с распределением вероятности по классам (подробнее [softmax function](http://cntk.ai/pythondocs/cntk.ops.html#cntk.ops.softmax) и другие функции активации [activation](https://cntk.ai/pythondocs/cntk.layers.layers.html#cntk.layers.layers.Activation) ).

## Обучение

Подобно CNTK 102, мы минимизируем кросс-энтропию - "cross-entropy" между меткой и прогнозируемой вероятностью сетью. Если эта терминология звучит странно для вас, обратитесь к CNTK 102 для обучения. Поскольку мы собираемся построить более одной модели, мы создадим несколько вспомогательных функций.

In [13]:
def create_criterion_function(model, labels):
    loss = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error(model, labels)
    return loss, errs # (model, labels) -> (loss, error metric)

Далее нам понадобится вспомогательная функция для обучения модели. Сначала создадим дополнительные вспомогательные функции, которые понадобятся для визуализации различных функций, связанных с обучением.

In [14]:
# Определите функцию полезности для вычисления скользящей средней суммы.
# Более эффективная реализация возможна с помощью функции np.cumsum()
def moving_average(a, w=5):
    if len(a) < w:
        return a[:]    # Нужно отправить копию массива
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]


# Определяет утилиту, которая печатает ход обучения
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))
        
    return mb, training_loss, eval_error

### Настройка обучения

В предыдущих уроках мы описали понятия функции `loss` , оптимизаторы или  [learners](https://cntk.ai/pythondocs/cntk.learners.html) и связанные с ними механизмы, необходимые для обучения модели. Пожалуйста, обратитесь к предыдущим учебным пособиям, чтобы получить описание этих концепций. В этом уроке мы комбинируем обучение модели и тестирование в вспомогательной функции ниже.


In [15]:
def train_test(train_reader, test_reader, model_func, num_sweeps_to_train_with=10):
    
    # Выполнить функцию модели; x - переменная ввода (функции)
    # Мы будем масштабировать пиксели входного изображения в диапазоне 0-1, разделив все входное значение на 255.
    model = model_func(x/255)
    
    # Выполнить функцию потери и ошибки
    loss, label_error = create_criterion_function(model, y)
    
    # Iсоздать экземпляр объекта тренера для обучения модели
    learning_rate = 0.2
    lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
    learner = C.sgd(z.parameters, lr_schedule)
    trainer = C.Trainer(z, (loss, label_error), [learner])
    
    # Инициализация параметров для тренера
    minibatch_size = 64
    num_samples_per_sweep = 60000
    num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size
    
    # Сопоставьте потоки данных со входом и метками.
    input_map={
        y  : train_reader.streams.labels,
        x  : train_reader.streams.features
    } 
    
    # Раскоммент ниже для более подробного ведения журнала
    training_progress_output_freq = 500
     
    # Запустить таймер
    start = time.time()

    for i in range(0, int(num_minibatches_to_train)):
        #Прочитайте мини-пакет из файла данных обучения
        data=train_reader.next_minibatch(minibatch_size, input_map=input_map) 
        trainer.train_minibatch(data)
        print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
     
    # Время обучения печати
    print("Training took {:.1f} sec".format(time.time() - start))
    
    # Проверить модель
    test_input_map = {
        y  : test_reader.streams.labels,
        x  : test_reader.streams.features
    }

    # Данные испытаний для обученной модели
    test_minibatch_size = 512
    num_samples = 10000
    num_minibatches_to_test = num_samples // test_minibatch_size

    test_result = 0.0   

    for i in range(num_minibatches_to_test):
    
        # Мы загружаем тестовые данные пакетами, указанными test_minibatch_size
        # EКаждая точка данных в minibatch представляет собой изображение цифры  
        # MNIST размером 784 с одним пикселем на измерение, которое мы будем 
        # кодировать / декодировать с помощью обучаемой модели.
        data = test_reader.next_minibatch(test_minibatch_size, input_map=test_input_map)
        eval_error = trainer.test_minibatch(data)
        test_result = test_result + eval_error

    # Среднее значение ошибки оценки всех тестовых minibatches
    print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))

### Запуск тренера и тестовой модели

Теперь мы готовы тренировать нашу сверточную нейронную сеть.

In [16]:
def do_train_test():
    global z
    z = create_model(x)
    reader_train = create_reader(train_file, True, input_dim, num_output_classes)
    reader_test = create_reader(test_file, False, input_dim, num_output_classes)
    train_test(reader_train, reader_test, z)
    
do_train_test()

NameError: name 'x' is not defined

Обратите внимание, что средняя ошибка теста очень сопоставима с нашей ошибкой обучения, указывающей на то, что наша модель имеет хорошую ошибку  "out of sample" error a.k.a. [generalization error](https://en.wikipedia.org/wiki/Generalization_error). Это означает, что наша модель может очень эффективно справляться с ранее невидимыми наблюдениями (во время учебного процесса). Это ключ, чтобы избежать [преобучения - overfitting](https://en.wikipedia.org/wiki/Overfitting).

Давайте посмотрим, что является значением некоторых параметров сети. Мы проверим значение смещения выходного плотного слоя. Раньше это было все 0. Теперь вы видите ненулевые значения, указывающие, что параметры модели были обновлены во время обучения.

In [17]:
print("Bias value of the last dense layer:", z.classify.b.value)

NameError: name 'z' is not defined

### Запуск оценки / прогнозирования
До сих пор мы имеем дело с совокупными мерами ошибки. Возьмем теперь вероятности, связанные с отдельными точками данных. Для каждого наблюдения функция `eval` возвращает распределение вероятности по всем классам. Классификатор обучается распознавать цифры, поэтому имеет 10 классов. Сначала проложим сетевой выход через функцию `softmax`. Это сопоставляет агрегированные активации по сети с вероятностями по 10 классам.

In [18]:
out = C.softmax(z)

NameError: name 'C' is not defined

Обратимся к небольшому образцу из тестовых данных.

In [20]:
# Прочтите данные для оценки
reader_eval=create_reader(test_file, False, input_dim, num_output_classes)

eval_minibatch_size = 25
eval_input_map = {x: reader_eval.streams.features, y:reader_eval.streams.labels} 

data = reader_eval.next_minibatch(eval_minibatch_size, input_map=eval_input_map)

img_label = data[y].asarray()
img_data = data[x].asarray()

# reshape img_data to: M x 1 x 28 x 28 to be compatible with model
img_data = np.reshape(img_data, (eval_minibatch_size, 1, 28, 28))

predicted_label_prob = [out.eval(img_data[i]) for i in range(len(img_data))]

In [19]:
# Найдите индекс с максимальным значением как для предсказанной, так и для основной истины
pred = [np.argmax(predicted_label_prob[i]) for i in range(len(predicted_label_prob))]
gtlabel = [np.argmax(img_label[i]) for i in range(len(img_label))]

NameError: name 'predicted_label_prob' is not defined

In [20]:
print("Label    :", gtlabel[:25])
print("Predicted:", pred)

NameError: name 'gtlabel' is not defined

Давайте визуализируем некоторые результаты

In [21]:
# Печать случайного изображения
sample_number = 5
plt.imshow(img_data[sample_number].reshape(28,28), cmap="gray_r")
plt.axis('off')

img_gt, img_pred = gtlabel[sample_number], pred[sample_number]
print("Image Label: ", img_pred)

NameError: name 'img_data' is not defined

## Слой объединения

Часто в некоторых случаях необходимо контролировать количество параметров, особенно при наличии глубоких сетей. Для каждого слоя выходного уровня свертки (каждый слой соответствует выходному сигналу фильтра), у него может быть слой объединения. Уровни объединения обычно вводятся в:
- Уменьшите размерность предыдущего слоя (ускорение сети),
- Делает модель более толерантной к изменениям в местоположении объекта на изображении. Например, даже когда цифра смещается в одну сторону изображения вместо того, чтобы быть посередине, классификатор будет выполнять задачу классификации хорошо.

Расчет на узле объединения намного проще, чем обычный упреждающий узел. Он не имеет веса, смещения или функции активации. Он использует простую функцию агрегации (например, max или average) для вычисления ее вывода. Наиболее часто используемой функцией является  "max" - максимальный узел объединения просто выводит максимум входных значений, соответствующих позиции фильтра на входе. На рисунке ниже показаны входные значения в области 4 x 4. Максимальный размер окна объединения составляет 2 x 2 и начинается с верхнего левого угла. Максимальное значение в окне становится выходом области. Каждый раз, когда модель сдвигается на величину, указанную параметром шага (как показано на рисунке ниже), и максимальная операция объединения повторяется. 
![maxppool](https://cntk.ai/jup/201/MaxPooling.png)

Другой альтернативой является средний пул, который испускает это среднее значение вместо максимального значения. Два разных пула кластера суммируются в анимации ниже.

In [22]:
# Графические изображения с шагом 2 и 1 с включенным дополнением
images = [("https://www.cntk.ai/jup/c103d_max_pooling.gif" , 'Max pooling'),
          ("https://www.cntk.ai/jup/c103d_average_pooling.gif", 'Average pooling')]

for im in images:
    print(im[1])
    display(Image(url=im[0], width=300, height=300))

Max pooling


NameError: name 'display' is not defined

## Типичная сеть свертки

![](http://www.cntk.ai/jup/conv103d_mnist-conv-mp.png)

Типичный CNN содержит набор чередующихся слоев свертки и объединения, за которыми следует плотный выходной уровень для классификации. Вы найдете варианты этой структуры во многих классических глубоких сетях (VGG, AlexNet и т.д.). Это контрастирует с сетью MLP, которую мы использовали в CNTK_103C, которая состояла из 2 плотных слоев, за которыми следовал плотный выходной уровень.

Иллюстрации представлены в контексте двумерных (2D) изображений, но концепция и компоненты CNTK могут работать с любыми размерными данными. На приведенной выше схеме показан 2 слоя свертки и 2 слоя с максимальным пулом. Типичная стратегия заключается в увеличении количества фильтров в более глубоких слоях при уменьшении пространственного размера каждого промежуточного слоя.

## Задание: Создайте сеть с помощью MaxPooling

Типичные сверточные сети имеют чередующиеся свертки и максимальные уровни пулов. В предыдущей модели был только слой свертки. В этом разделе вы создадите модель со следующей архитектурой.

![](https://www.cntk.ai/jup/cntk103d_conv_max2.png)

Для достижения этой цели вы будете использовать функцию CNTK [MaxPooling](https://cntk.ai/pythondocs/cntk.layers.layers.html#cntk.layers.layers.MaxPooling) . Вы отредактируете функцию `create_model` ниже и добавьте операцию.

Подсказка: мы предлагаем решение для нескольких ячеек ниже. Не смотрите вперед и старайтесь сначала добавить слой.

In [23]:
# Изменить эту модель
def create_model(features):
    with C.layers.default_options(init = C.glorot_uniform(), activation = C.relu):
            h = features
            
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(2,2), 
                                       pad=True, name='first_conv')(h)
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=16, 
                                       strides=(2,2), 
                                       pad=True, name='second_conv')(h)
            r = C.layers.Dense(num_output_classes, activation = None, name='classify')(h)
            return r
        
# do_train_test()

**Вопрос**: Сколько у нас параметров в модели с MaxPooling и Convolution? Какая из двух моделей дает более низкий коэффициент ошибок?


**Самостоятельная работа**
- Помогает ли использование LeakyRelu повысить коэффициент ошибок?
- Какой процент параметра делает последний плотный слой вкладом w.r.t. общее число параметров для (а) чисто двух сверточных слоев и (б) чередующихся 2 сверточных и максимальных слоев   

## Решение

In [24]:
# функция построения модели
def create_model(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):
            h = features
            
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(1,1), 
                                       pad=True, name="first_conv")(h)
            h = C.layers.MaxPooling(filter_shape=(2,2), 
                                    strides=(2,2), name="first_max")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=16, 
                                       strides=(1,1), 
                                       pad=True, name="second_conv")(h)
            h = C.layers.MaxPooling(filter_shape=(3,3), 
                                    strides=(3,3), name="second_max")(h)
            r = C.layers.Dense(num_output_classes, activation = None, name="classify")(h)
            return r
        
do_train_test()

NameError: name 'x' is not defined